In [1]:
import transformers

/Users/agnieszka.dul/Library/Caches/pypoetry/virtualenvs/ds-learning-Qd2jz2Gq-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

Text generation - learning the model

## Data

In [28]:
1+1

2

## Experiment

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)


/Users/agnieszka.dul/Library/Caches/pypoetry/virtualenvs/ds-learning-Qd2jz2Gq-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [11]:
input = 'cat is sitting on a sunny'
token = tokenizer(input, return_tensors = "pt")
token

{'input_ids': tensor([[ 9246,   318,  5586,   319,   257, 27737]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [7]:
tokenizer.decode(319)

' on'

In [27]:
output = model(**token)
output.logits

tensor([[[ -30.4126,  -29.6744,  -32.4685,  ...,  -38.5661,  -37.2363,
           -30.6164],
         [-103.6783, -104.1460, -107.0027,  ..., -112.3242, -109.4508,
          -106.8523],
         [-104.9496, -106.9342, -113.1350,  ..., -117.2863, -111.3649,
          -109.4713],
         [ -74.3458,  -74.0159,  -80.5104,  ...,  -82.2178,  -81.3614,
           -76.4972],
         [ -95.7849,  -93.9787,  -98.1079,  ..., -102.1896,  -98.6558,
           -95.2035],
         [-109.4507, -105.7443, -114.2843,  ..., -118.0063, -113.5199,
          -109.7081]]], grad_fn=<UnsafeViewBackward0>)

In [16]:
output.logits[0,-1].argmax()

tensor(11)

In [17]:
tokenizer.decode(11)

','

In [20]:
out = output.logits[0,-1].argsort()[-10:][:,None]

In [21]:
tokenizer.batch_decode(out)

[' bench',
 ' summer',
 ' hill',
 ' morning',
 ' patio',
 ' afternoon',
 ' side',
 ' day',
 ' beach',
 ',']

In [22]:
out = model.generate(**token)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/agnieszka.dul/Library/Caches/pypoetry/virtualenvs/ds-learning-Qd2jz2Gq-py3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[ 9246,   318,  5586,   319,   257, 27737,    11, 27737,  1110,    11,
           290,   339,   338,   655,   546,   284,   467,   284,  3996,    13]])

In [26]:
tokenizer.decode(out[0])

"cat is sitting on a sunny, sunny day, and he's just about to go to bed."